#**ITCS4156-Introduction to Machine Lerning**

#**Classification Assignment**


Your Name: **Hritika Kucheriya**


In [1]:
# pip install wget



In [2]:
import os
import random
import traceback
from pdb import set_trace
import sys
import numpy as np
from abc import ABC, abstractmethod
import traceback
from scipy.stats import norm  # Add missing import for norm distribution



In [3]:
from util.timer import Timer
from util.data import split_data, dataframe_to_array, binarize_classes
from util.metrics import accuracy
from sklearn.metrics import confusion_matrix
from datasets.MNISTDataset import MNISTDataset



# Understand MNIST Dataset


![](https://camo.githubusercontent.com/01c057a753e92a9bc70b8c45d62b295431851c09cffadf53106fc0aea7e2843f/687474703a2f2f692e7974696d672e636f6d2f76692f3051493378675875422d512f687964656661756c742e6a7067)

The dataset you'll be using for this project is the famous [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset which contains images of handwritten digits 0 through 9. There are 60,000 images included in the dataset and each image is a gray scale image of size 28x28. Each pixel represents a feature which means there are $28*28$ or $784$ features per each data sample.

**The goal of the dataset is to classify each image of handwritten digits correctly!**

The dataset consists of 3 splits:

1. **Train**: Throughout this assignment you will be training your model using this data. There are approximately 44k training samples.
2. **Validation**: You will then use this set to tune your model and evaluate its performance. There are approximately 12k training samples.
3. **Test**: This split simulates real life data which we often don't have access to until the model is deployed. We have kept this split hidden from you and we will use it to judge the performance of your model on Autolab.

You DO NOT have access to the Test set as it gonna be used for scoring. This will not prevent you to complete this assignment at all.


# Design Machine Learning Models (TODO)


## Base Model
Basic model structure, **don't change** this component.


In [4]:
class BaseModel(ABC):
    """ Super class for ITCS Machine Learning Class"""

    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass



## ClassificationModel 
Basic model structure, **don't change** this component.


In [5]:
class ClassificationModel(BaseModel):
    """
        Abstract class for classification 
        
        Attributes
        ==========
    """

    # check if the matrix is 2-dimensional. if not, raise an exception    
    def _check_matrix(self, mat, name):
        if len(mat.shape) != 2:
            raise ValueError(f"Your matrix {name} shape is not 2D! Matrix {name} has the shape {mat.shape}")
        
    ####################################################
    #### abstract funcitons ############################
    @abstractmethod
    def fit(self, X: np.ndarray, y: np.ndarray):
        """
            train classification model
            
            Args:
                X:  Input data
                
                y:  targets/labels
        """        
        pass
    
    @abstractmethod
    def predict(self, X: np.ndarray):
        """
            apply the learned model to input X
            
            parameters
            ----------
            X     2d array
                  input data
            
        """        
        pass 




## TODO: Classification with Perceptron
*Please complete the TODOs. *


In [6]:
class Perceptron(ClassificationModel):
    """
        Performs Gaussian Naive Bayes
    
        attributes:
            alpha: learning rate or step size used by gradient descent.
                
            epochs (int): Number of times data is used to update the weights `self.w`.
                Each epoch means a data sample was used to update the weights at least
                once.
                
            seed (int): Seed to be used for NumPy's RandomState class
                or universal seed np.random.seed() function.
            
            batch_size (int): Mini-batch size used to determine the size of mini-batches
                if mini-batch gradient descent is used.
            
            w (np.ndarray): NumPy array which stores the learned weights.
    """
    def __init__(self, alpha: float, epochs: int = 1, seed: int = None):
        ClassificationModel.__init__(self)
        self.alpha = alpha
        self.epochs = epochs
        self.seed = seed
        self.w = None
        
    def fit(self, X: np.ndarray, y: np.ndarray):
        """ Train model to learn optimal weights when performing binary classification.
        
            Args:
                X: Data 
                
                y: Targets/labels
                
             TODO:
                Finish this method by using Rosenblatt's Perceptron algorithm to learn
                the best weights to classify the binary data. There is no need to
                implement th pocket algorithm unless you choose to do so. Also, update 
                and store the learned weights into `self.w`.
        """
        
        if self.seed is not None:
             np.random.seed(self.seed)
        
        # Preprocess data - add bias term
        X_bias = np.c_[np.ones((X.shape[0], 1)), X]
        
        # Better weight initialization
        self.w = np.zeros((X_bias.shape[1], 1))
        
        # Format labels to -1 and 1 for easier computation
        y_formatted = np.where(y == 0, -1, 1).reshape(-1, 1)
        
        # Implement averaged perceptron
        avg_w = np.zeros_like(self.w)
        best_w = None
        best_accuracy = 0
        
        # Initial learning rate
        alpha = self.alpha
        
        # Counter for averaging
        c = 1
        
        for epoch in range(self.epochs):
            # Learning rate decay with a slower decay rate
            current_alpha = alpha / (1 + 0.001 * epoch)
            
            # Shuffle the data for each epoch
            indices = np.random.permutation(X_bias.shape[0])
            X_shuffled = X_bias[indices]
            y_shuffled = y_formatted[indices]
            
            errors = 0
            
            for i in range(X_shuffled.shape[0]):
                # Calculate net input
                net_input = np.dot(X_shuffled[i], self.w)
                
                # Check if prediction is correct
                if y_shuffled[i] * net_input <= 0:
                    # Update weights
                    update = current_alpha * y_shuffled[i] * X_shuffled[i].reshape(-1, 1)
                    self.w += update
                    errors += 1
                
                # Update average weights
                avg_w += self.w
                c += 1
            
            # Early stopping if no errors
            if errors == 0 and epoch > 10:
                break
            
            # Evaluate current model
            predictions = np.sign(np.dot(X_bias, self.w))
            predictions[predictions <= 0] = 0  # Convert -1 to 0
            current_accuracy = np.mean(predictions == y)
            
            # Save best model
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                best_w = self.w.copy()
        
        # Use averaged weights or best weights, whichever is better
        avg_w = avg_w / c
        
        # Evaluate averaged model
        avg_predictions = np.sign(np.dot(X_bias, avg_w))
        avg_predictions[avg_predictions <= 0] = 0  # Convert -1 to 0
        avg_accuracy = np.mean(avg_predictions == y)
        
        # Choose the better model
        if avg_accuracy > best_accuracy:
            self.w = avg_w
        else:
            self.w = best_w

    def predict(self, X: np.ndarray):
        """ Make predictions using the learned weights.
        
            Args:
                X: Data 

            TODO:
                Finish this method by adding code to make a prediction given the learned
                weights `self.w`. Store the predicted labels into `y_hat`.
        """
        # TODO Add code below
        
        X_bias = np.c_[np.ones((X.shape[0], 1)), X]
        
        net_input = np.dot(X_bias, self.w)
        
        
        y_hat = np.ones([len(X), 1]) # TODO Store predictions here by replacing np.ones()
        # Makes sure predictions are given as a 2D array

        y_hat = np.where(net_input >= 0, 1, 0)
        return y_hat.reshape(-1, 1)





## TODO: Classification with NaiveBayes
*Please complete the TODOs. *


In [7]:
class NaiveBayes(ClassificationModel):
    """
        Performs Gaussian Naive Bayes
    
        attributes:
            smoothing: smoothing hyperparameter used to prevent numerical instability and 
                divide by zero errors
                
            class_labels (np.ndarray or list): Unique labels for the passed data. This 
                should be set in the fit() method.
            
            priors (np.ndarray): NumPy array which stores the priors.
            
            log_priors (np.ndarray): NumPy array which stores the log of the priors and
                used by the predict() method.
            
            means (np.ndarray): NumPy array of means used by the
                log_gaussian_distribution() method to compute the log likelihoods
            
            stds (np.ndarray): NumPy array of standard deviations used by the
                log_gaussian_distribution() method to compute the log likelihoods
            
            feature_importance (np.ndarray): NumPy array of feature importance scores
    """
    def __init__(self, smoothing: float = 10e-3):
        ClassificationModel.__init__(self)
        self.smoothing = smoothing
        # All class variables that need to be set somewhere within the below methods.
        self.class_labels = None
        self.priors = None
        self.log_priors = None
        self.means = None
        self.stds = None
        self.feature_importance = None  # Store feature importance scores

    def log_gaussian_distribution(self, X: np.ndarray, mu: np.ndarray, std: np.ndarray) -> np.ndarray:
        """ Computes the log of a value at a given point in a Gaussian distribution
        
            Args:
                X: Data for which an output value is computed for.
                
                mu: Feature means
                
                var: Feature variance
        """
        # Improved numerical stability
        variance = std**2
        # Avoid very small variances that could cause numerical issues
        variance = np.maximum(variance, 1e-10)
        return -0.5 * np.log(2 * np.pi * variance) - 0.5 * ((X - mu)**2) / variance
        
    def compute_priors(self, y: np.ndarray) -> None:
        """ Computes the priors and log priors for each class.
    
            Args:
                y: Lables
                
            TODO: 
                Finish this method by computing the priors and log priors to be used when
                making predictions using MAP. Store the computed priors and log priors 
                into self.priors and self.log_priors.
                
        """
        # TODO Add code below
        
        self.class_labels = np.unique(y)
        
        
        class_counts = np.zeros(len(self.class_labels))
        for i, label in enumerate(self.class_labels):
            class_counts[i] = np.sum(y == label)
        
        # Apply Laplace smoothing to the priors for better balance
        class_counts += 1  # Add 1 to each class count (Laplace smoothing)
        
        # Compute priors with smoothing
        self.priors = class_counts / np.sum(class_counts)
        self.log_priors = np.log(self.priors)

    def compute_parameters(self, X: np.ndarray, y: np.ndarray) -> None:
        """ Computes the means and standard deviations for classes and features
        
            Args:
                X: Data 
                
                y: Targets/labels

            TODO: 
                Finish this method by computing the means and stds for the Gaussian
                distribution which will then be used to comput the likelihoods. Store
                the computed means and stds into self.means and self.stds.
        """
        # TODO Add code below
        
        n_features = X.shape[1]
        n_classes = len(self.class_labels)
        
        self.means = np.zeros((n_classes, n_features))
        self.stds = np.zeros((n_classes, n_features))
        
        # Ensure y is flattened to avoid shape mismatch
        y = y.flatten()
        
        # Calculate feature importance based on variance between classes
        self.feature_importance = np.zeros(n_features)
        
        # First compute means for each class
        class_means = np.zeros((n_classes, n_features))
        for i, label in enumerate(self.class_labels):
            class_samples = X[y == label]
            if len(class_samples) > 0:
                class_means[i] = np.mean(class_samples, axis=0)
        
        # Compute feature importance as the variance of means between classes
        # Features with higher variance between class means are more discriminative
        for j in range(n_features):
            self.feature_importance[j] = np.var(class_means[:, j])
        
        # Normalize feature importance
        if np.sum(self.feature_importance) > 0:
            self.feature_importance = self.feature_importance / np.sum(self.feature_importance)
        
        for i, label in enumerate(self.class_labels):
            # Get samples for this class
            class_samples = X[y == label]
            
            if len(class_samples) > 0:
                # Compute mean and std for each feature
                self.means[i] = np.mean(class_samples, axis=0)
                # Use a more robust std calculation
                self.stds[i] = np.std(class_samples, axis=0) + self.smoothing
            else:
                # Handle the case where a class has no samples
                self.means[i] = np.zeros(n_features)
                self.stds[i] = np.ones(n_features) * self.smoothing

    def compute_log_likelihoods(self, X: np.ndarray) -> np.ndarray:
        """ Computes and returns log likelihoods using the means and stds
                
            Args:
                X: Data 
        
            TODO:
                Finish this method by computing the log likelihoods of the passed data 
                `X`. Use the `self.means` and `self.stds` class variables you set in 
                the `compute_parameters()` method along with the `log_gaussian_distribution()` 
                method which is defined for you. The `log_gaussian_distribution()`  
                will apply the log to your feature likelihoods for you so you don't need to!
                This method should return the computed log likelihoods.
        """
        
        n_samples = X.shape[0]
        n_classes = len(self.class_labels)
        log_likelihoods = np.zeros((n_samples, n_classes))
        
        # Optimized implementation with feature importance weighting
        for i in range(n_classes):
            # Pre-compute log-likelihoods for each feature and class
            feature_log_likelihoods = np.zeros((n_samples, X.shape[1]))
            
            for j in range(X.shape[1]):
                # Weight the log-likelihood by feature importance
                feature_weight = 1.0
                if self.feature_importance is not None:
                    feature_weight = 1.0 + 9.0 * self.feature_importance[j]  # Scale between 1 and 10
                
                feature_log_likelihoods[:, j] = feature_weight * self.log_gaussian_distribution(
                    X[:, j], self.means[i, j], self.stds[i, j]
                )
            
            # Sum log-likelihoods across features for each sample
            log_likelihoods[:, i] = np.sum(feature_log_likelihoods, axis=1)
        
        return log_likelihoods

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """ Computes the priors and Gaussian parameters used for predicting.
        
            Args:
                X: Data 
                
                y: Targets/labels
                
             TODO:
                Finish this method by computing the priors and Gaussian parameters. 
                To do so, first finish and then call the compute_parameters() and 
                compute_priors() methods.
        """
        self.class_labels = np.unique(y)
        # TODO Add code below
        
        self.compute_parameters(X, y)
        self.compute_priors(y)

    def predict(self, X) -> np.ndarray:
        """ Comptues a prediction using log likelihoods and log priors.
        
            Args:
                X: Data 
        
             TODO:
                Finish this method by computing the log likelihoods and log priors.
                To do so, first finishing and then call the compute_log_likelihoods() 
                method. You'll also need to access the class variables self.log_priors 
                and self.class_labels you set when running the fit(), compute_parameters() 
                and compute_priors() methods. Store the predicted labels into `y_hat`.
        """
        # TODO Add code below
        
        log_likelihoods = self.compute_log_likelihoods(X)
        
        posterior_log_probs = log_likelihoods + self.log_priors
        
        predicted_class_indices = np.argmax(posterior_log_probs, axis=1)
        
        # Map indices to actual class labels
        y_hat = self.class_labels[predicted_class_indices]
        
        # Makes sure predictions are given as a 2D array
        return y_hat.reshape(-1, 1)




## TODO: Classification with Logistic Regression
*Please complete the TODOs. *


In [8]:
class LogisticRegression(ClassificationModel):
    """
        Performs Logistic Regression using the softmax function.
    
        attributes:
            alpha: learning rate or step size used by gradient descent.
                
            epochs: Number of times data is used to update the weights `self.w`.
                Each epoch means a data sample was used to update the weights at least
                once.
            
            seed (int): Seed to be used for NumPy's RandomState class
                or universal seed np.random.seed() function.
            
            batch_size: Mini-batch size used to determine the size of mini-batches
                if mini-batch gradient descent is used.
            
            w (np.ndarray): NumPy array which stores the learned weights.
    """
    def __init__(self, alpha: float, epochs: int = 1,  seed: int = None, batch_size: int = None):
        ClassificationModel.__init__(self)
        self.alpha = alpha
        self.epochs = epochs
        self.seed = seed
        self.batch_size = batch_size
        self.w = None
        self.reg_lambda = 0.01  # L2 regularization parameter

    def softmax(self, z: np.ndarray) -> np.ndarray:
        """ Computes probabilities for multi-class classification given continuous inputs z.
        
            Args:
                z: Continuous outputs after dotting the data with the current weights 

            TODO:
                Finish this method by adding code to return the softmax. Don't forget
                to subtract the max from `z` to maintain  numerical stability!
        """
        
        # Improved numerical stability by subtracting max value
        z_stable = z - np.max(z, axis=1, keepdims=True)
        
        # Clip values to prevent overflow/underflow
        z_stable = np.clip(z_stable, -500, 500)
        
        exp_z = np.exp(z_stable)
        
        # Add small epsilon to denominator for numerical stability
        return exp_z / (np.sum(exp_z, axis=1, keepdims=True) + 1e-10)

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        """ Train our model to learn optimal weights for classifying data.
        
            Args:
                X: Data 
                
                y: Targets/labels
                
             TODO:
                Finish this method by using either batch or mini-batch gradient descent
                to learn the best weights to classify the data. You'll need to finish and 
                also call the `softmax()` method to complete this method. Also, update 
                and store the learned weights into `self.w`. 
        """
         # Initialize weights
        if self.seed is not None:
             np.random.seed(self.seed)
         
        
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
         
        
        self.w = np.random.randn(n_features, n_classes) * 0.01
         
        
        y_one_hot = np.zeros((n_samples, n_classes))
        for i in range(n_samples):
             y_one_hot[i, int(y[i])] = 1
         
        # Learning rate schedule
        initial_alpha = self.alpha
        
        for epoch in range(self.epochs):
            # Implement learning rate decay
            current_alpha = initial_alpha / (1 + 0.01 * epoch)
            
            if self.batch_size is None:
                
                z = X @ self.w
                y_hat = self.softmax(z)
                
                
                # Add L2 regularization term to gradient
                gradient = X.T @ (y_hat - y_one_hot) / n_samples + self.reg_lambda * self.w
                
               
                self.w -= current_alpha * gradient
            else:
                
                indices = np.random.permutation(n_samples)
                for start_idx in range(0, n_samples, self.batch_size):
                    batch_indices = indices[start_idx:start_idx + self.batch_size]
                    X_batch = X[batch_indices]
                    y_batch = y_one_hot[batch_indices]
                    
                    
                    z_batch = X_batch @ self.w
                    y_hat_batch = self.softmax(z_batch)
                    
                    
                    # Add L2 regularization term to gradient
                    gradient = X_batch.T @ (y_hat_batch - y_batch) / len(batch_indices) + self.reg_lambda * self.w
                    
                   
                    self.w -= current_alpha * gradient

       
    def predict(self, X: np.ndarray):
        """ Make predictions using the learned weights.
        
            Args:
                X: Data 

            TODO:
                Finish this method by adding code to make a prediction given the learned
                weights `self.w`. Store the predicted labels into `y_hat`.
        """
        # TODO Add code below

        z = X @ self.w
        
        y_proba = self.softmax(z)
        
        # Get the class with the highest probability
        y_hat = np.argmax(y_proba, axis=1)

        # Makes sure predictions are given as a 2D array
        return y_hat.reshape(-1, 1)




## TODO: Define Hyperparameters 
*Please complete the TODOs. *


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

class HyperParametersAndTransforms():
    
    @staticmethod
    def get_params(name):
        model = getattr(HyperParametersAndTransforms, name)
        params = {}
        for key, value in model.__dict__.items():
            if not key.startswith('__') and not callable(key):
                if not callable(value) and not isinstance(value, staticmethod):
                    params[key] = value
        return params
    
    class Perceptron():
        """Kwargs for classifier the Perceptron class and data prep"""
        model_kwargs = dict(
            alpha = 1.0,  # Higher learning rate
            epochs = 2000,  # More epochs for thorough training
            seed = 42, # Setting seed for reproducible results
        )

        data_prep_kwargs = dict(
            target_pipe = Pipeline([
                ('identity', 'passthrough')
            ]),
            
            feature_pipe = Pipeline([
                ('scaler', StandardScaler())
            ])
        )
        
    class NaiveBayes():
        """Kwargs for classifier the NaiveBayes class and data prep"""
        model_kwargs = dict(
            smoothing = 5e-3,  # Increased smoothing parameter for better generalization
        )
        
        data_prep_kwargs = dict(
            target_pipe = Pipeline([
                ('identity', 'passthrough')
            ]),
            feature_pipe = Pipeline([
                ('scaler', StandardScaler())
            ])
        )
        
    class LogisticRegression():
        model_kwargs = dict(
            alpha = 0.1,    # Increased learning rate for faster convergence
            epochs = 300,   # Increased number of training iterations for better convergence
            seed = 42,      # Setting seed for reproducible results
            batch_size = 256,  # Larger batch size for better gradient estimation
        )
      
        data_prep_kwargs = dict(
            target_pipe = Pipeline([
                ('identity', 'passthrough')
            ]),
            
            feature_pipe = Pipeline([
                ('scaler', StandardScaler())
            ])
        )




# Model Training and Testing


## Define Preprocessing Functions


In [10]:
from abc import ABC, abstractmethod

class DataPreparation():
    def __init__(self, target_pipe, feature_pipe):
        self.target_pipe = target_pipe
        self.feature_pipe = feature_pipe
        
    @abstractmethod
    def data_prep(self):
        pass
    
    def fit(self, X, y=None):
        if self.target_pipe  is not None:
            self.target_pipe.fit(y)
            
        if self.feature_pipe is not None:
            self.feature_pipe.fit(X)

    def transform(self, X, y=None):
        if self.target_pipe is not None:
            y = self.target_pipe.transform(y)
            
        if self.feature_pipe is not None:
            X = self.feature_pipe.transform(X)

        return X, y
    
    def fit_transform(self, X, y):
        self.fit(X, y)
        X, y = self.transform(X, y)
        return X, y



In [11]:
class MNISTDataPreparation(DataPreparation):
    def __init__(self, target_pipe, feature_pipe):
        super().__init__(target_pipe, feature_pipe)
        
    def data_prep(self, binarize=False, return_array=False):
        mnist_dataset = MNISTDataset()
        X_trn_df, y_trn_df, X_vld_df, y_vld_df = mnist_dataset.load()
        
        # Converts MNIST problem to classifying ONLY 1s vs 0s
        if binarize:
            X_trn_df, y_trn_df = binarize_classes(
                X_trn_df, 
                y_trn_df, 
                pos_class=[1],
                neg_class=[0], 
            )
            
            X_vld_df, y_vld_df = binarize_classes(
                X_vld_df, 
                y_vld_df, 
                pos_class=[1], 
                neg_class=[0], 
            )

        X_trn_df, y_trn_df = self.fit_transform(X=X_trn_df, y=y_trn_df)
        X_vld_df, y_vld_df = self.transform(X=X_vld_df, y=y_vld_df)

        if return_array:
            print("Returning data as NumPy array...")
            return dataframe_to_array([X_trn_df, y_trn_df, X_vld_df, y_vld_df])
            
        return X_trn_df, y_trn_df, X_vld_df, y_vld_df



## Define Model running (training/fit and testing/evaluate)


In [12]:
def get_name(obj):
    try:
        if hasattr(obj, '__name__'):
            return obj.__name__
        else:
            return obj
    except Exception as e:
        return obj
    
def catch_and_throw(e, err):
    trace = traceback.format_exc()
    print(err + f"\n{trace}")
    raise e



In [13]:
class RunModel():
    t1 = '\t'
    t2 = '\t\t'
    t3 = '\t\t\t'
    def __init__(self, model, model_params):
        self.model_name = model.__name__
        self.model_params = model_params
        self.model = self.build_model(model, model_params)

    def build_model(self, model, model_params):
        print("="*50)
        print(f"Building model {self.model_name}")
        
        try:
            model = model(**model_params)
        except Exception as e:
            err = f"Exception caught while building model for {self.model_name}:"
            catch_and_throw(e, err)
        return model
    
    def fit(self, *args, **kwargs):
        print(f"Training {self.model_name}...")
        print(f"{self.t1}Using hyperparameters: ")
        [print(f"{self.t2}{n} = {get_name(v)}")for n, v in self.model_params.items()]
        try: 
            return self._fit(*args, **kwargs)
        except Exception as e:
            err = f"Exception caught while training model for {self.model_name}:"
            catch_and_throw(e, err)
            
    def _fit(self, X, y, metrics=None, pass_y=False):
        if pass_y:
            self.model.fit(X, y)
        else:
             self.model.fit(X)
        preds = self.model.predict(X)
        scores = self.get_metrics(y, preds, metrics, prefix='Train')
        return scores
    
    def evaluate(self, *args, **kwargs):
        print(f"Evaluating {self.model_name}...")
        try:
            return self._evaluate(*args, **kwargs)
        except Exception as e:
            err = f"Exception caught while evaluating model for {self.model_name}:"
            catch_and_throw(e, err)
        

    def _evaluate(self, X, y, metrics, prefix=''):
        preds = self.model.predict(X)
        scores = self.get_metrics(y, preds, metrics, prefix)      
        return scores
    
    def predict(self, X):
        try:
            preds = self.model.predict(X)
        except Exception as e:
            err = f"Exception caught while making predictions for model {self.model_name}:"
            catch_and_throw(e, err)
            
        return preds
    
    def get_metrics(self, y, y_hat, metrics, prefix=''):
        scores = {}
        for name, metric in metrics.items():
            score = metric(y, y_hat)
            display_score = round(score, 3)
            scores[name] = score
            print(f"{self.t2}{prefix} {name}: {display_score}")
        return scores



In [14]:
def run_eval(eval_stage='validation'):
    main_timer = Timer()
    main_timer.start()

    total_points = 0
    
    task_info = [
       dict(
            model=Perceptron,
            name='Perceptron',
            data=MNISTDataPreparation,
            data_prep=dict(binarize=True, return_array=True),
            metrics=dict(acc=accuracy),
            eval_metric='acc',
            rubric=rubric_perceptron,
            trn_score=0,
            eval_score=0,
            successful=False,
        ),
        dict(
            model=NaiveBayes,
            name='NaiveBayes',
            data=MNISTDataPreparation,
            data_prep=dict(return_array=True),
            metrics=dict(acc=accuracy),
            eval_metric='acc',
            rubric=rubric_naive_bayes,
            trn_score=0,
            eval_score=0,
            successful=False,
        ),
        dict(
            model=LogisticRegression,
            name='LogisticRegression',
            data=MNISTDataPreparation,
            data_prep=dict(return_array=True),
            metrics=dict(acc=accuracy),
            rubric=rubric_logistic_regression,
            eval_metric='acc',
            trn_score=0,
            eval_score=0,
            successful=False,
        ),
    ]
    
    total_points = 0
    
    for info in task_info:
        task_timer =  Timer()
        task_timer.start()
        try:
            params = HyperParametersAndTransforms.get_params(info['name'])
            model_kwargs = params.get('model_kwargs', {})
            data_prep_kwargs = params.get('data_prep_kwargs', {})
            
            run_model = RunModel(info['model'], model_kwargs)
            data = info['data'](**data_prep_kwargs)
            X_trn, y_trn, X_vld, y_vld = data.data_prep(**info['data_prep'])
            
            trn_scores = run_model.fit(X_trn, y_trn, info['metrics'], pass_y=True)
            eval_scores = run_model.evaluate(X_vld, y_vld, info['metrics'], prefix=eval_stage.capitalize())
            
            info['trn_score'] = trn_scores[info['eval_metric']]
            info['eval_score'] = eval_scores[info['eval_metric']]
            info['successful'] = True
                
        except Exception as e:
            track = traceback.format_exc()
            print("The following exception occurred while executing this test case:\n", track)
        task_timer.stop()
        
        print("")
        points = info['rubric'](info['eval_score'])
        print(f"Points Earned: {points}")
        total_points += points
        
    print("="*50)
    print('')
    main_timer.stop()
    
    avg_trn_acc, avg_eval_acc, successful_tests = summary(task_info)
    task_eval_acc = get_eval_scores(task_info)
    total_points = int(round(total_points))
    
    print(f"Tests passed: {successful_tests}/{ len(task_info)}, Total Points: {total_points}/80\n")
    print(f"Average Train Accuracy: {avg_trn_acc}")
    print(f"Average {eval_stage.capitalize()} Accuracy: {avg_eval_acc}")
    
    return (total_points, avg_eval_acc, main_timer.last_elapsed_time, avg_trn_acc, *task_eval_acc)

def summary(task_info):
    sum_trn_acc = 0
    sum_eval_acc = 0
    successful_tests = 0

    for info in task_info:
        if info['successful']:
            successful_tests += 1
            sum_trn_acc += info['trn_score']
            sum_eval_acc += info['eval_score']
    
    if successful_tests == 0:
        return 0, 0, successful_tests
    
    avg_trn_acc = sum_trn_acc / len(task_info)
    avg_eval_acc = sum_eval_acc / len(task_info)
    return round(avg_trn_acc, 4), round(avg_eval_acc, 4), successful_tests

def get_eval_scores(task_info):
    return [i['eval_score'] for i in task_info]



## Evaluation Related Functions
Don't change this section.


In [15]:
def rubric_perceptron(acc, max_score=25):
    score_percent = 0
    if acc >= 0.95:
        score_percent = 100
    elif acc >= 0.90:
        score_percent = 90
    elif acc >= 0.80:
        score_percent = 80
    elif acc >= 0.70:
        score_percent = 70
    elif acc >= 0.60:
        score_percent = 60
    elif acc >= 0.50:
        score_percent = 50
    else:
        score_percent = 40
    score = max_score * score_percent / 100.0 
    return score

def rubric_naive_bayes(acc, max_score=25):
    score_percent = 0
    if acc >= 0.75:
        score_percent = 100
    elif acc >= 0.65:
        score_percent = 90
    elif acc >= 0.55:
        score_percent = 80
    elif acc >= 0.40:
        score_percent = 70
    elif acc >= 0.30:
        score_percent = 60
    elif acc >= 0.20:
        score_percent = 50
    elif acc >= 0.10:
        score_percent = 45
    else:
        score_percent = 40
    score = max_score * score_percent / 100.0 
    return score
   
def rubric_logistic_regression(acc, max_score=30):
    score_percent = 0
    if acc >= 0.85:
        score_percent = 100
    elif acc >= 0.80:
        score_percent = 90
    elif acc >= 0.75:
        score_percent = 80
    elif acc >= 0.70:
        score_percent = 70
    elif acc >= 0.60:
        score_percent = 60
    elif acc >= 0.50:
        score_percent = 55
    elif acc >= 0.40:
        score_percent = 50
    elif acc >= 0.30:
        score_percent = 45
    else:
        score_percent = 40
    score = max_score * score_percent / 100.0 
    return score



# Test your code
Run the following cell to test your code (or for **debugging**).


In [16]:
if __name__ == "__main__":
    run_eval()



Building model Perceptron
Skipping download. File already exists: /Users/som/Downloads/Classification Mini Project/datasets/data/MNIST/train.zip

Unzipping: /Users/som/Downloads/Classification Mini Project/datasets/data/MNIST/train.zip

Skipping download. File already exists: /Users/som/Downloads/Classification Mini Project/datasets/data/MNIST/val.zip

Unzipping: /Users/som/Downloads/Classification Mini Project/datasets/data/MNIST/val.zip

Loading dataset with Pandas...
Done!
Returning data as NumPy array...
Training Perceptron...
	Using hyperparameters: 
		alpha = 1.0
		epochs = 2000
		seed = 42
		Train acc: 0.53
Evaluating Perceptron...
		Validation acc: 0.53
Elapsed time: 2.1317 seconds

Points Earned: 12.5
Building model NaiveBayes
Skipping download. File already exists: /Users/som/Downloads/Classification Mini Project/datasets/data/MNIST/train.zip

Unzipping: /Users/som/Downloads/Classification Mini Project/datasets/data/MNIST/train.zip

Skipping download. File already exists: /Us

/var/folders/yp/wyy5kprd443_5jd6zdmkbp800000gn/T/ipykernel_23684/1187489158.py:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_one_hot[i, int(y[i])] = 1


		Train acc: 0.925
Evaluating LogisticRegression...
		Validation acc: 0.915
Elapsed time: 62.2804 seconds

Points Earned: 30.0

Elapsed time: 68.8132 seconds
Tests passed: 3/3, Total Points: 62/80

Average Train Accuracy: 0.6784
Average Validation Accuracy: 0.6744


In [17]:
# %%